In [3]:
# 4_train_naive_bayes.py
# Huấn luyện Naive Bayes Classifier

import pandas as pd
import json
import pickle
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [4]:
# Load dữ liệu
print("📂 Đang load dữ liệu...")
df = pd.read_csv("../data/processed/naive_bayes_data.csv")

# Load symptom mapping
with open("../data/processed/symptom_mapping.json", "r", encoding="utf-8") as f:
    symptom_mapping = json.load(f)

all_symptoms = symptom_mapping["all_symptoms"]

# Tách features và labels
X = df[all_symptoms].values
y = df["disease"].values

print(f"✓ Đã load {len(X)} mẫu dữ liệu")
print(f"✓ Số lượng features (triệu chứng): {len(all_symptoms)}")
print(f"✓ Số lượng classes (bệnh): {len(np.unique(y))}")

📂 Đang load dữ liệu...
✓ Đã load 139 mẫu dữ liệu
✓ Số lượng features (triệu chứng): 219
✓ Số lượng classes (bệnh): 37


In [5]:
# Phân tích phân bố mẫu theo từng bệnh
from collections import Counter
disease_counts = Counter(y)
print(f"\n📊 Phân bố mẫu theo từng bệnh:")
for disease, count in sorted(disease_counts.items(), key=lambda x: (x[1], x[0])):
    print(f"   - {disease}: {count} mẫu")

# Lọc bỏ các class có ít hơn 2 mẫu (cần cho stratify)
min_samples_per_class = 2
classes_to_keep = [disease for disease, count in disease_counts.items() if count >= min_samples_per_class]
classes_to_remove = [disease for disease, count in disease_counts.items() if count < min_samples_per_class]

if classes_to_remove:
    print(f"\n⚠️  Loại bỏ {len(classes_to_remove)} bệnh có ít hơn {min_samples_per_class} mẫu:")
    for disease in sorted(classes_to_remove):
        print(f"   - {disease}: {disease_counts[disease]} mẫu")
    
    # Lọc dữ liệu
    mask = np.isin(y, classes_to_keep)
    X = X[mask]
    y = y[mask]
    print(f"\n✓ Sau khi lọc: {len(X)} mẫu, {len(classes_to_keep)} bệnh")

# Chia train/test với stratify (chỉ khi mỗi class có ít nhất 2 mẫu)
try:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print(f"\n📊 Chia dữ liệu (với stratify):")
except ValueError as e:
    print(f"\n⚠️  Lỗi khi chia dữ liệu với stratify: {e}")
    print(f"   → Chuyển sang chia dữ liệu không dùng stratify")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=None
    )

print(f"   - Train: {len(X_train)} mẫu")
print(f"   - Test: {len(X_test)} mẫu")


📊 Phân bố mẫu theo từng bệnh:
   - DISEASE: 1 mẫu
   - Loãng xương: 2 mẫu
   - Mề đay: 2 mẫu
   - Nhiễm trùng đường tiết niệu: 2 mẫu
   - Thủy đậu: 2 mẫu
   - Viêm bàng quang: 2 mẫu
   - Viêm gan B: 2 mẫu
   - Viêm tai giữa: 2 mẫu
   - Viêm tụy cấp: 2 mẫu
   - Zona (Giời leo): 2 mẫu
   - Bệnh trĩ: 3 mẫu
   - Gout (Gút): 3 mẫu
   - Mất ngủ: 3 mẫu
   - Rối loạn lo âu: 3 mẫu
   - Suy giáp: 3 mẫu
   - Thiếu máu: 3 mẫu
   - Tiểu đường (Type 2): 3 mẫu
   - Trầm cảm: 3 mẫu
   - Tăng huyết áp: 3 mẫu
   - Cường giáp (Basedow): 4 mẫu
   - Eczema (Chàm): 4 mẫu
   - Hen suyễn: 4 mẫu
   - Sỏi thận: 4 mẫu
   - Trào ngược dạ dày (GERD): 4 mẫu
   - Viêm họng: 4 mẫu
   - Viêm khớp dạng thấp: 4 mẫu
   - Viêm ruột thừa: 4 mẫu
   - Đau mắt đỏ: 4 mẫu
   - Cảm lạnh: 5 mẫu
   - Sốt xuất huyết: 5 mẫu
   - Dị ứng: 6 mẫu
   - Viêm xoang: 6 mẫu
   - Đau nửa đầu: 6 mẫu
   - Nhiễm khuẩn đường ruột: 7 mẫu
   - Viêm dạ dày: 7 mẫu
   - Viêm phổi: 7 mẫu
   - Cúm: 8 mẫu

⚠️  Loại bỏ 1 bệnh có ít hơn 2 mẫu:
   - DISEAS

In [6]:
# Training Naive Bayes
print("\n🚀 Bắt đầu training Naive Bayes...")
model = MultinomialNB(alpha=1.0)  # alpha: Laplace smoothing
model.fit(X_train, y_train)

# Đánh giá trên tập test
y_pred = model.predict(X_test)
accuracy = model.score(X_test, y_test)

print(f"\n✓ Hoàn thành training!")
print(f"✓ Accuracy: {accuracy:.2%}")


🚀 Bắt đầu training Naive Bayes...

✓ Hoàn thành training!
✓ Accuracy: 21.43%


e:\CHATBOT\training\venv\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")


In [7]:

# Classification report
print("\n📊 Chi tiết đánh giá:")
print(classification_report(y_test, y_pred, zero_division=0))

# Confusion matrix
print("\n📊 Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
# Lấy các class xuất hiện trong test set (có thể khác với tất cả class trong y)
test_classes = sorted(np.unique(np.concatenate([y_test, y_pred])))
cm_df = pd.DataFrame(cm, index=test_classes, columns=test_classes)
print(cm_df)

# Lưu model
print("\n💾 Đang lưu model...")
with open("../models/naive_bayes_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Lưu thêm thông tin cần thiết
# Lấy tất cả class có trong training set
diseases = sorted(np.unique(y_train))
model_info = {
    "symptoms": all_symptoms,
    "diseases": diseases,
    "accuracy": float(accuracy)
}

with open("../models/model_info.json", "w", encoding="utf-8") as f:
    json.dump(model_info, f, ensure_ascii=False, indent=2)

print("✓ Đã lưu 'naive_bayes_model.pkl'")
print("✓ Đã lưu 'model_info.json'")

# Test với mẫu mới
print("\n" + "="*60)
print("🧪 TEST VỚI DỮ LIỆU MỚI")
print("="*60)

def predict_disease(symptoms_list, model, all_symptoms):
    """Dự đoán bệnh từ danh sách triệu chứng"""
    # Chuyển sang binary vector
    vector = [1 if symptom in symptoms_list else 0 for symptom in all_symptoms]
    vector = np.array(vector).reshape(1, -1)
    
    # Dự đoán
    prediction = model.predict(vector)[0]
    probabilities = model.predict_proba(vector)[0]
    
    # Lấy top 3 bệnh có xác suất cao nhất
    # Sử dụng model.classes_ để đảm bảo đúng với thứ tự class trong model
    top_indices = np.argsort(probabilities)[::-1][:3]
    top_diseases = [(model.classes_[i], probabilities[i]) for i in top_indices]
    
    return prediction, top_diseases

# Test cases
test_cases = [
    ["sốt", "đau_đầu", "ho"],
    ["tiêu_chảy", "đau_bụng"],
    ["đau_đầu", "nghẹt_mũi", "chảy_nước_mũi"],
]

for i, symptoms in enumerate(test_cases, 1):
    print(f"\nTest case {i}:")
    print(f"   Triệu chứng: {', '.join(symptoms)}")
    
    pred, top_diseases = predict_disease(symptoms, model, all_symptoms)
    
    print(f"   ➜ Chẩn đoán: {pred}")
    print(f"   ➜ Top 3 khả năng:")
    for disease, prob in top_diseases:
        print(f"      - {disease}: {prob:.2%}")

print("\n✅ Hoàn tất!")


📊 Chi tiết đánh giá:
                          precision    recall  f1-score   support

                     Cúm       0.07      1.00      0.12         1
    Cường giáp (Basedow)       0.00      0.00      0.00         3
                Cảm lạnh       0.00      0.00      0.00         1
                  Dị ứng       0.00      0.00      0.00         0
           Eczema (Chàm)       1.00      1.00      1.00         1
              Gout (Gút)       0.00      0.00      0.00         1
               Hen suyễn       0.00      0.00      0.00         1
  Nhiễm khuẩn đường ruột       0.33      1.00      0.50         1
          Rối loạn lo âu       0.00      0.00      0.00         2
                Suy giáp       0.00      0.00      0.00         1
                Sỏi thận       1.00      0.50      0.67         2
          Sốt xuất huyết       0.00      0.00      0.00         1
               Thiếu máu       0.00      0.00      0.00         0
                Thủy đậu       0.00      0.00      0.

e:\CHATBOT\training\venv\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
e:\CHATBOT\training\venv\Lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
e:\CHATBOT\training\venv\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
e:\CHATBOT\training\venv\Lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes 